# MSc Thesis - Duygu Ider - NLP SECTION

## BERT-Based Classifiers - Fine-Tuning and Predicting Using Crypto Data

This is the NLP-related section of my thesis. BERT-Based classifiers are fine-tuned using the news and social media data about cryptocurrencies. Sentiment classification is performed.

## Intro

Install and load libraries

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:
pip install transformers datasets ray

     |████████████████████████████████| 3.8 MB 5.0 MB/s 
     |████████████████████████████████| 311 kB 60.2 MB/s 
     |████████████████████████████████| 59.6 MB 73 kB/s 
     |████████████████████████████████| 6.5 MB 46.1 MB/s 
     |████████████████████████████████| 596 kB 27.3 MB/s 
     |████████████████████████████████| 67 kB 3.9 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 212 kB 56.8 MB/s 
     |████████████████████████████████| 134 kB 57.9 MB/s 
     |████████████████████████████████| 1.1 MB 30.8 MB/s 
     |████████████████████████████████| 175 kB 34.7 MB/s 
     |████████████████████████████████| 144 kB 42.1 MB/s 
     |████████████████████████████████| 94 kB 1.2 MB/s 
     |████████████████████████████████| 271 kB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import torch
from transformers import (BertForSequenceClassification, AdamW, BertConfig, AutoModelForSequenceClassification, 
                          AutoTokenizer, Trainer, TrainingArguments)
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import numpy as np
import random
from transformers import get_linear_schedule_with_warmup
import time
import datetime
from datasets import list_datasets, load_dataset, Dataset, load_metric
from pprint import pprint
from transformers import Trainer, TrainingArguments
from google.colab import files, drive

## Data

Connect to Drive

In [ ]:
drive.mount('/drive')

Mounted at /drive


Load train and test dataset

In [ ]:
train = pd.read_csv("/drive/My Drive/Colab Notebooks/train_crypto.csv",sep='\t')
valid = pd.read_csv("/drive/My Drive/Colab Notebooks/validation_crypto.csv",sep='\t')

In [ ]:
test = pd.read_excel("/drive/My Drive/Colab Notebooks/remainder_data_daily_100.xlsx", index_col=0)

In [ ]:
test['label'] = 0

In [ ]:
test

,source,cryptocurrency,date,text,label
0,news,BTC,2019-08-01,Fundstrat Co-Founder Tom Lee: US Fed Rate Cuts...,0
1,news,BTC,2019-08-01,Andorra Telecom Inks New Partnership to Run Bl...,0
2,news,BTC,2019-08-01,Application of Transaction Mining in Online Ga...,0
3,news,BTC,2019-08-01,"Blockchain Adoption Takes Off in Airlines, Avi...",0
4,news,BTC,2019-08-01,Kraken Exchange Acquires Service Provider for ...,0
...,...,...,...,...,...
182927,news,ETH,2022-02-21,Sipher’s Alain Dinh on What’s Next for NFT Gam...,0
182928,news,ETH,2022-02-21,Pocket Network becomes primary provider for Fu...,0
182929,news,ETH,2022-02-21,"$250K bounty 'not too low to be insulting,' sa...",0
182930,news,ETH,2022-02-21,NFTs and social capital: How projects are coll...,0


In [ ]:
train

,Unnamed: 0,text,label
0,0,#Bitcoin is the language of money. One day bil...,neutral
1,1,"My bank called me today to tell me they ""don't...",negative
2,2,Ruffer Turns Bitcoin Investment Into $750M Pro...,positive
3,3,Next BoE Governor: ‘Be Prepared to Lose All Yo...,negative
4,4,Very much worth reading.\n\n#Bitcoin https://t...,neutral
...,...,...,...
4812,4812,$150 GIVEAWAY ~ 24 HOURS ⏳ \n\n- RT &amp; foll...,neutral
4813,4813,My own issue is that I don't understand bitcoi...,negative
4814,4814,Brave’s International Referral Program Hits Ma...,negative
4815,4815,"Retail FOMO coming in, Will push this to NEW A...",positive


In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import recall_score, precision_score, f1_score

## Setup

Define the labels as positive, negative, neutral

In [ ]:
def num_labels(data):
    
    data = np.where(data == "positive", 2, data)
    data = np.where(data == "neutral", 1, data)
    data = np.where(data == "negative", 0, data)
    
    return data.astype(int)



Define the BERT tokenizer for tokenization of the text samples as appropriate input for BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7

Input the word list for BERT-Context (not used for the other BERT classifiers)

In [ ]:
word_list = ['bitcoin', 'crypto', 'eth', 'ethereum', 'btc', 'blockchain', 'defi', 'nft', 'binance',
             'cryptocurrency', 'bullish']
print(word_list)

['bitcoin', 'crypto', 'eth', 'ethereum', 'btc', 'blockchain', 'defi', 'nft', 'binance', 'cryptocurrency', 'bullish']


Add these special tokens for BERT-Context only

In [ ]:
special_tokens_dict = {'additional_special_tokens':word_list}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

Assigning ['bitcoin', 'crypto', 'eth', 'ethereum', 'btc', 'blockchain', 'defi', 'nft', 'binance', 'cryptocurrency', 'bullish'] to the additional_special_tokens key of the tokenizer
Adding bitcoin to the vocabulary
Adding crypto to the vocabulary
Adding eth to the vocabulary
Adding ethereum to the vocabulary
Adding btc to the vocabulary
Adding blockchain to the vocabulary
Adding defi to the vocabulary
Adding nft to the vocabulary
Adding binance to the vocabulary
Adding cryptocurrency to the vocabulary
Adding bullish to the vocabulary


Test whether special tokens work or not

In [ ]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("bitcoin is a cryptocurrency"))

['[CLS]', 'bitcoin', 'is', 'not', 'a', 'shit', '##co', '##in', '[SEP]']

Check number of tokens in the vocab list of BERT

In [ ]:
len(tokenizer)

30533

Encode the train and valid datasets, initialize the model, define compute metrics

In [ ]:
train_encodings = tokenizer(train['text'].tolist(), truncation=True, padding='max_length', max_length=32)
valid_encodings = tokenizer(valid['text'].tolist(), truncation=True, padding='max_length', max_length=32)
train_labels = num_labels(train.label).tolist()
valid_labels = num_labels(valid.label).tolist()

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_labels)
valid_dataset = TextDataset(valid_encodings, valid_labels)
metric = load_metric("accuracy")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3,   
    output_attentions = False, output_hidden_states = False)

#for param in model.base_model.parameters():
#    param.requires_grad = False

#for name, param in model.named_parameters():
#    print(name, param.requires_grad)
    

def model_init():
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3,   
        output_attentions = False, output_hidden_states = False)
    for param in model.base_model.parameters():
        param.requires_grad = False
    model.resize_token_embeddings(len(tokenizer))
    return model

def model_init_unfrozen():
    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 3,   
        output_attentions = False, output_hidden_states = False)
    model.resize_token_embeddings(len(tokenizer))
    return model

#def compute_metrics(eval_pred):
#    predictions, labels = eval_pred
#    predictions = predictions.argmax(axis=-1)
#    return metric.compute(predictions=predictions, references=labels)

def compute_metrics(p):    
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1} 

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size":

## Train

In [ ]:
#CUDA_LAUNCH_BLOCKING=1

Define the trainer and training arguments for model fitting

In [ ]:

training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy="epoch",     
    learning_rate=1e-2,
    #learning_rate=2e-5,
    seed=24,
    num_train_epochs=4,             
    per_device_train_batch_size=32,  
    warmup_steps=0,               
    weight_decay=0.01,              
    save_total_limit=1,                
)




trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    model_init=model_init_unfrozen,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from al

This part only fits the model using the given hyperparameters above

In [ ]:
start = time.time()
trainer.train()
end = time.time()
print("elapsed time is ", round((end - start)/60), " minutes")

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size":

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.114186,0.963485,0.963485,0.963485,0.963485
2,No log,0.121548,0.970124,0.970124,0.970124,0.970124
3,No log,0.090230,0.980083,0.980083,0.980083,0.980083
4,0.117900,0.101457,0.978423,0.978423,0.978423,0.978423


***** Running Evaluation *****
  Num examples = 1205
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1205
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1205
  Batch size = 8
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
added tokens file saved in ./results/checkpoint-500/added_tokens.json
Deleting older checkpoint [results/checkpoint-1500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1205
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




elapsed time is  2  minutes


Evaluate the model performance

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1205
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.9800829875518672,
 'eval_f1': 0.9800829875518672,
 'eval_loss': 0.08778780698776245,
 'eval_precision': 0.9800829875518672,
 'eval_recall': 0.9800829875518672,
 'eval_runtime': 4.731,
 'eval_samples_per_second': 254.702,
 'eval_steps_per_second': 31.917}

In [ ]:
import os
#os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

Convert to string

In [ ]:
test['text'] = test['text'].apply(lambda x: str(x))

Predict on test data

In [ ]:
test_encodings = tokenizer(test['text'].tolist(), truncation=True, padding='max_length', max_length=32)
test_labels = test.label.tolist() #num_labels(test.prediction).tolist()
test_dataset = TextDataset(test_encodings, test_labels)

raw_pred, _, _ = trainer.predict(test_dataset)
y_pred = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 182932
  Batch size = 8


In [ ]:
test['pred_bert_newtokens'] = y_pred

In [ ]:
data = data.drop(['label'], axis=1)

In [ ]:
#test = pd.read_excel('/drive/My Drive/Colab Notebooks/test_crypto_optimized_combined.xlsx', index_col=0)

In [ ]:
test

,source,cryptocurrency,date,text,label,pred_bert_fro,pred_bert_unfro,pred_bert_newtokens
0,news,BTC,2019-08-01,Fundstrat Co-Founder Tom Lee: US Fed Rate Cuts...,0,0,2,2
1,news,BTC,2019-08-01,Andorra Telecom Inks New Partnership to Run Bl...,0,2,2,2
2,news,BTC,2019-08-01,Application of Transaction Mining in Online Ga...,0,2,1,1
3,news,BTC,2019-08-01,"Blockchain Adoption Takes Off in Airlines, Avi...",0,2,2,2
4,news,BTC,2019-08-01,Kraken Exchange Acquires Service Provider for ...,0,2,2,2
...,...,...,...,...,...,...,...,...
182927,news,ETH,2022-02-21,Sipher’s Alain Dinh on What’s Next for NFT Gam...,0,2,1,1
182928,news,ETH,2022-02-21,Pocket Network becomes primary provider for Fu...,0,2,2,2
182929,news,ETH,2022-02-21,"$250K bounty 'not too low to be insulting,' sa...",0,0,0,0
182930,news,ETH,2022-02-21,NFTs and social capital: How projects are coll...,0,2,1,2


In [ ]:
print("Accuracy - ZSC & FinBERT Labels:", accuracy_score(num_labels(test.label).tolist(), y_pred))

Accuracy - ZSC Labels: 0.7144329896907217
Accuracy - Actual Labels: 0.6876288659793814


In [ ]:
drive.mount('/drive')

Mounted at /drive


In [ ]:
test.to_excel('/drive/My Drive/Colab Notebooks/crypto_daily_100_3.xlsx', sheet_name = 'test_output')

In [ ]:
#trainer.save_model('/drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens')

Saving model checkpoint to /drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens
Configuration saved in /drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens/config.json
Model weights saved in /drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens/pytorch_model.bin
tokenizer config file saved in /drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens/tokenizer_config.json
Special tokens file saved in /drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens/special_tokens_map.json
added tokens file saved in /drive/My Drive/Colab Notebooks/unfrozen_crypto_newtokens/added_tokens.json


Load a saved model that I have fit before

In [ ]:
model = BertForSequenceClassification.from_pretrained("/drive/My Drive/Colab Notebooks/unfrozen_crypto", local_files_only=True)
#trainer = Trainer(model=model)
#trainer.model = model.cuda()
trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    model=model,
    compute_metrics=compute_metrics)

loading configuration file /drive/My Drive/Colab Notebooks/unfrozen_crypto/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading

RuntimeError: ignored

## Hyperparam Tuning

Hyperparameter tuning for BERT models

In [ ]:
import ray
from ray import tune
from ray.tune.schedulers import PopulationBasedTraining, MedianStoppingRule
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter

In [ ]:
pip install -U tensorboardx

     |████████████████████████████████| 125 kB 4.1 MB/s 


Define the hyperparameter space to fit it using GPU

In [ ]:
def hp_space_fn(empty_arg):
    config = {
                "learning_rate": tune.loguniform(5e-6, 1e-4), #3e-4, 1e-4, 5e-5, 3e-5
                #"learning_rate": tune.loguniform(1e-3, 5e-1), #3e-4, 1e-4, 5e-5, 3e-5
                "num_train_epochs": tune.choice(range(2, 6)),
                "seed": tune.choice(range(1, 41)),
                "per_device_train_batch_size": tune.choice([8, 16, 32, 64]),
                #"learning_rate": tune.choice([5e-6, 2e-5, 3e-5, 2e-5]),
                #"learning_rate": tune.choice([1e-2, 3e-2, 5e-2, 8e-3, 1e-1]),
                "weight_decay": tune.quniform(0.0, 0.15, 0.01),
    }
    wandb_config = {
            "wandb": {
                    "project": os.environ.get(
                        'WANDB_PROJECT',
                        'wandb_project'),
                    "api_key": os.environ.get('API_KEY'),
                    "log_config": True
                    }
    }
    config.update(wandb_config)
    return config

In [ ]:
import os

Perform hyperparam search and fit optimal model

In [ ]:
start = time.time()
best_run = trainer.hyperparameter_search(n_trials=20, direction="maximize",
                                         hp_space=hp_space_fn, resources_per_trial={"cpu": 2, "gpu": 1})
                                   
end = time.time()
print("elapsed time is ", round((end - start)/60), " minutes")

Output hidden; open in https://colab.research.google.com to view.

Best params

In [ ]:
best_run.hyperparameters

{'learning_rate': 3.607459472971186e-05,
 'num_train_epochs': 4,
 'per_device_train_batch_size': 32,
 'seed': 20,
 'wandb': {'api_key': None, 'log_config': True, 'project': 'wandb_project'},
 'weight_decay': 0.03}

Best scores

In [ ]:
print(best_run)

BestRun(run_id='61580_00019', objective=3.933609958506224, hyperparameters={'learning_rate': 3.607459472971186e-05, 'num_train_epochs': 4, 'seed': 20, 'per_device_train_batch_size': 32, 'weight_decay': 0.03, 'wandb': {'project': 'wandb_project', 'api_key': None, 'log_config': True}})


## PBT

PBT tuning

In [ ]:
start = time.time()
best_run = trainer.hyperparameter_search(n_trials=15,
                                         direction="maximize",
                                         scheduler=PopulationBasedTraining(
                                                    time_attr="training_iteration",
                                                    metric='eval_accuracy',
                                                    mode='max',
                                                    perturbation_interval=10
                                            #        hyperparam_mutations={
                                             #           "learning_rate": tune.loguniform(1e-6, 1e-4),
                                              #          "num_train_epochs": tune.choice(range(1, 6)),
                                               #         "seed": tune.choice(range(1, 41)),
                                                #        "per_device_train_batch_size": tune.choice([8, 16, 32, 64]),
                                                 #       #"learning_rate": tune.choice([5e-6, 2e-5, 3e-5, 2e-5]),
                                                  #      #"learning_rate": tune.choice([1e-2, 3e-2, 5e-2, 8e-3, 1e-1]),
                                                   #     "weight_decay": tune.quniform(0.0, 0.15, 0.01),
                                                    #}
                                                ),
                                         hp_space=hp_space_fn,
                                         keep_checkpoints_num=1,
                                         checkpoint_score_attr='eval_accuracy',
                                         reuse_actors=True,
                                         resources_per_trial={"cpu": 2, "gpu": 1})
end = time.time()
print("elapsed time is ", round((end - start)/60), " minutes")

2022-02-28 11:24:22,570	WARNING trial_runner.py:1454 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0
2022-02-28 11:24:22,574	WARNING trial_runner.py:1454 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A more strict API access pattern would be enforced starting 1.12s.0
2022-02-28 11:24:22,922	WARNING trial_runner.py:1454 -- You are trying to access _search_alg interface of TrialRunner in TrialScheduler, which is being restricted. If you believe it is reasonable for your scheduler to access this TrialRunner API, please reach out to Ray team on GitHub. A m

== Status ==
Current time: 2022-02-28 11:24:23 (running for 00:00:00.65)
Memory usage on this node: 7.1/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/2 CPUs, 0/1 GPUs, 0.0/4.28 GiB heap, 0.0/2.14 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-02-28_11-24-22
Number of trials: 15/15 (15 PENDING)
+------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------+----------------+
| Trial name             | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |   weight_decay |
|------------------------+----------+-------+-----------------+--------------------+-------------------------------+--------+----------------|
| _objective_fa61d_00000 | PENDING  |       |     8.3183e-05  |                  2 |                            16 |     24 |           0.05 |
| _objective_fa61d_00001 | PENDING  |       |     

(_objective pid=2081) Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
(_objective pid=2081) - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2081) - This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=2081) Some we

== Status ==
Current time: 2022-02-28 11:24:51 (running for 00:00:29.32)
Memory usage on this node: 9.5/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/4.28 GiB heap, 0.0/2.14 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-02-28_11-24-22
Number of trials: 15/15 (14 PENDING, 1 RUNNING)
+------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+--------+----------------+
| Trial name             | status   | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |   weight_decay |
|------------------------+----------+-----------------+-----------------+--------------------+-------------------------------+--------+----------------|
| _objective_fa61d_00000 | RUNNING  | 172.28.0.2:2081 |     8.3183e-05  |                  2 |                            16 |     24 |           0.

 23%|██▎       | 89/388 [00:08<00:29, 10.29it/s]


...

== Status ==
Current time: 2022-02-28 11:39:31 (running for 00:15:08.57)
Memory usage on this node: 9.9/12.7 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 2.0/2 CPUs, 1.0/1 GPUs, 0.0/4.28 GiB heap, 0.0/2.14 GiB objects (0.0/1.0 accelerator_type:P100)
Result logdir: /root/ray_results/_objective_2022-02-28_11-24-22
Number of trials: 15/15 (1 RUNNING, 14 TERMINATED)
+------------------------+------------+-----------------+-----------------+--------------------+-------------------------------+--------+----------------+-------------+
| Trial name             | status     | loc             |   learning_rate |   num_train_epochs |   per_device_train_batch_size |   seed |   weight_decay |   objective |
|------------------------+------------+-----------------+-----------------+--------------------+-------------------------------+--------+----------------+-------------|
| _objective_fa61d_00014 | RUNNING    | 172.28.0.2:2081 |     1.26299e-06 |                  2 | 

  0%|          | 0/97 [00:00<?, ?it/s]
(_objective pid=2081) 
 10%|█         | 10/97 [00:00<00:00, 89.69it/s]
(_objective pid=2081) 
 20%|█▉        | 19/97 [00:00<00:00, 80.04it/s]
(_objective pid=2081) 
 29%|██▉       | 28/97 [00:00<00:00, 77.72it/s]
(_objective pid=2081) 
 38%|███▊      | 37/97 [00:00<00:00, 78.64it/s]
(_objective pid=2081) 
 46%|████▋     | 45/97 [00:00<00:00, 77.99it/s]
(_objective pid=2081) 
 55%|█████▍    | 53/97 [00:00<00:00, 78.04it/s]
(_objective pid=2081) 
 63%|██████▎   | 61/97 [00:00<00:00, 75.08it/s]
(_objective pid=2081) 
 71%|███████   | 69/97 [00:00<00:00, 75.90it/s]
(_objective pid=2081) 
 79%|███████▉  | 77/97 [00:00<00:00, 76.40it/s]
(_objective pid=2081) 
 88%|████████▊ | 85/97 [00:01<00:00, 77.23it/s]
(_objective pid=2081) 
 96%|█████████▌| 93/97 [00:01<00:00, 77.37it/s]
                                                 
100%|██████████| 776/776 [00:59<00:00, 13.14it/s]


Result for _objective_fa61d_00014:
  date: 2022-02-28_11-39-35
  done: false
  epoch: 2.0
  eval_accuracy: 0.586340206185567
  eval_f1: 0.586340206185567
  eval_loss: 0.8729087710380554
  eval_precision: 0.586340206185567
  eval_recall: 0.586340206185567
  eval_runtime: 1.2637
  eval_samples_per_second: 614.074
  eval_steps_per_second: 76.759
  experiment_id: 73247fa208434601a4cea3181efa4ec1
  hostname: ae023b7c350a
  iterations_since_restore: 2
  node_ip: 172.28.0.2
  objective: 2.345360824742268
  pid: 2081
  time_since_restore: 62.08331561088562
  time_this_iter_s: 32.081539154052734
  time_total_s: 62.08331561088562
  timestamp: 1646048375
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: fa61d_00014
  
Result for _objective_fa61d_00014:
  date: 2022-02-28_11-39-35
  done: true
  epoch: 2.0
  eval_accuracy: 0.586340206185567
  eval_f1: 0.586340206185567
  eval_loss: 0.8729087710380554
  eval_precision: 0.586340206185567
  eval_recall: 0.586340206185567
  eval_runtime

2022-02-28 11:39:35,469	INFO tune.py:636 -- Total run time: 912.95 seconds (912.58 seconds for the tuning loop).


elapsed time is  15  minutes


## Zero-Shot Classifier

Predict using zero-shot classifier BART MNLI

In [ ]:
from transformers import pipeline

load BART

In [ ]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)
https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpy4oolvp3


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0342e09
creating metadata file for /root/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0342e09
loading configuration file https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/980f2be6bd282c5079e99199d7554cfd13000433ed0fdc527e7def799e5738fe.4fdc7ce6768977d347b32986aff152e26fcebbda34ef89ac9b114971d0342e09
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-mnli",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForSequenceClassification"
  ],
  "attention

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/35014754ae1fcb956d44903df02e4f69d0917cab0901ace5ac7f4a4a998346fe.a30bb5d685bb3c6e9376ab4480f1b252d9796d438d1c84a9b2deb0275c5b2151
creating metadata file for /root/.cache/huggingface/transformers/35014754ae1fcb956d44903df02e4f69d0917cab0901ace5ac7f4a4a998346fe.a30bb5d685bb3c6e9376ab4480f1b252d9796d438d1c84a9b2deb0275c5b2151
loading weights file https://huggingface.co/facebook/bart-large-mnli/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/35014754ae1fcb956d44903df02e4f69d0917cab0901ace5ac7f4a4a998346fe.a30bb5d685bb3c6e9376ab4480f1b252d9796d438d1c84a9b2deb0275c5b2151
All model checkpoint weights were used when initializing BartForSequenceClassification.

All the weights of BartForSequenceClassification were initialized from the model checkpoint at facebook/bart-large-mnli.
If your task is similar to the task the model 

...

storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/b4f8395edd321fd7cd8a87bca767b1135680a41d8931516dd1a447294633b9db.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
creating metadata file for /root/.cache/huggingface/transformers/b4f8395edd321fd7cd8a87bca767b1135680a41d8931516dd1a447294633b9db.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
https://huggingface.co/facebook/bart-large-mnli/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpjrkuj0n8


Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/19c09c9654551e163f858f3c99c226a8d0026acc4935528df3b09179204efe4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/19c09c9654551e163f858f3c99c226a8d0026acc4935528df3b09179204efe4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/facebook/bart-large-mnli/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmph2d8m7wx


Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-large-mnli/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/540455855ce0a3c13893c5d090d142de9481365bd32dc5457c957e5d13444d23.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/540455855ce0a3c13893c5d090d142de9481365bd32dc5457c957e5d13444d23.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/facebook/bart-large-mnli/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/b4f8395edd321fd7cd8a87bca767b1135680a41d8931516dd1a447294633b9db.647b4548b6d9ea817e82e7a9231a320231a1c9ea24053cc9e758f3fe68216f05
loading file https://huggingface.co/facebook/bart-large-mnli/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/19c09c9654551e163f858f3c99c226a8d0026acc4935528df3b09179204efe4c.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading

Define labels

In [ ]:
the_labels = ["positive", "negative"]

In [ ]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [ ]:
results=[]
scores=[]

Predict

In [ ]:
for idx, item in tqdm(test.iterrows()):
  res = classifier(test['text'], the_labels, multi_label=True)
  results.append(res['labels'][0])  
  scores.append(res['scores'][0])

10it [00:22,  2.26s/it]


KeyboardInterrupt: ignored

In [ ]:
res

{'labels': ['positive', 'negative'],
 'scores': [0.9723063111305237, 0.000790437392424792],
 'sequence': 'Fundstrat Co-Founder Tom Lee: US Fed Rate Cuts Are Helping Bitcoin'}